In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical, plot_model

In [ ]:
dataset = mnist.load_data()
(X_train, y_train), (X_test, y_test) = dataset

print("shape of X =", X_train.shape)
print("shape of y =", y_train.shape)

11490434/11490434 [==============================] - 1s 0us/step
shape of X = (60000, 28, 28)
shape of y = (60000,)


In [ ]:
# Flatten the inputs
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Convert the target to one-hot encoding
number_of_classes = 10
y_train = to_categorical(y_train, num_classes=number_of_classes)
y_test = to_categorical(y_test, num_classes=number_of_classes)


print("shape of X =", X_train.shape)
print("shape of y =", y_train.shape)

shape of X = (60000, 784)
shape of y = (60000, 10)


# Install the keras-tuner library

In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
def build_model(hp):
    model = Sequential()
    model.add( Input(shape = (784,)) )
    number_of_layers = hp.Int("num_layers", 1, 3)
    for i in range(number_of_layers):
      units = hp.Int(f"units_{i}", min_value=5, max_value=20, step=5)
      activation = hp.Choice("activation", ["relu", "tanh", "sigmoid"])
      model.add( Dense(units, activation=activation) ) #hidden layer

    model.add( Dense(10, activation='softmax') ) #output layer, activation must be softmax

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
    )
    return model

In [ ]:
import keras_tuner

# build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=1,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 20, 'step': 5, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}


In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 42s]
val_accuracy: 0.7555999755859375

Best val_accuracy So Far: 0.836899995803833
Total elapsed time: 00h 01m 50s


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 1
units_0: 10
activation: sigmoid
Score: 0.836899995803833

Trial 2 summary
Hyperparameters:
num_layers: 3
units_0: 10
activation: relu
units_1: 5
units_2: 5
Score: 0.7555999755859375

Trial 0 summary
Hyperparameters:
num_layers: 1
units_0: 5
activation: relu
Score: 0.11349999904632568


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1, 'units_0': 10, 'activation': 'sigmoid'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                7850      
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
Total params: 7,960
Trainable params: 7,960
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=20, initial_epoch=5, validation_data=(X_test, y_test))

Epoch 6/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5919 - accuracy: 0.8264 - val_loss: 0.5629 - val_accuracy: 0.8359
Epoch 7/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5861 - accuracy: 0.8277 - val_loss: 0.5513 - val_accuracy: 0.8328
Epoch 8/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.5550 - accuracy: 0.8379 - val_loss: 0.5102 - val_accuracy: 0.8562
Epoch 9/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5404 - accuracy: 0.8442 - val_loss: 0.5431 - val_accuracy: 0.8377
Epoch 10/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5340 - accuracy: 0.8430 - val_loss: 0.5115 - val_accuracy: 0.8467
Epoch 11/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5404 - accuracy: 0.8400 - val_loss: 0.5644 - val_accuracy: 0.8278
Epoch 12/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5400 - accuracy: 0.8442 - val_loss: 0.5676 - val_accura